In [2]:
%pip install opytimark
%pip install opytimizer
%pip install opfython
%pip install scikit-learn



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
import opfython.math.general as g
import opfython.stream.splitter as s
from opfython.models.supervised import SupervisedOPF
from sklearn.datasets import load_digits

from opytimizer import Opytimizer
from opytimizer.core import Function
from opytimizer.optimizers.swarm import PSO
from opytimizer.spaces import BooleanSpace

# Loads dataset
digits = load_digits()

# Gathers samples and targets
X = digits.data
Y = digits.target

#Adding 1 to labels, i.e., OPF should have labels from 1+
Y += 1

#Splits data into training and testing data
X_train, X_test, Y_train, Y_test = s.split(X, Y, percentage=0.5, random_state=1)

opf = SupervisedOPF(distance="log_squared_euclidean", pre_computed_distance=None)

opf.fit(X_train, Y_train)

# Predict databases

preds = opf.predict(X_test)

acc = g.opf_accuracy(Y_test, preds)

print(f"ACURÁCIA: {acc}")

2024-08-24 14:16:28,422 - opfython.stream.splitter — INFO — Splitting data ...
2024-08-24 14:16:28,423 - opfython.stream.splitter — DEBUG — X_1: (898, 64) | X_2: (899, 64) | Y_1: (898,) | Y_2: (899,).
2024-08-24 14:16:28,424 - opfython.stream.splitter — INFO — Data splitted.
2024-08-24 14:16:28,425 - opfython.models.supervised — INFO — Overriding class: OPF -> SupervisedOPF.
2024-08-24 14:16:28,425 - opfython.core.opf — INFO — Creating class: OPF.
2024-08-24 14:16:28,426 - opfython.core.opf — DEBUG — Distance: log_squared_euclidean | Pre-computed distance: False.
2024-08-24 14:16:28,426 - opfython.core.opf — INFO — Class created.
2024-08-24 14:16:28,426 - opfython.models.supervised — INFO — Class overrided.
2024-08-24 14:16:28,427 - opfython.models.supervised — INFO — Fitting classifier ...


2024-08-24 14:16:28,434 - opfython.models.supervised — DEBUG — Finding prototypes ...
2024-08-24 14:16:29,145 - opfython.models.supervised — DEBUG — Prototypes: [834, 65, 378, 333, 708, 892, 820, 523, 576, 113, 592, 37, 221, 743, 609, 104, 648, 157, 326, 353, 450, 66, 220, 515, 6, 683, 116, 451, 337, 281, 713, 540, 759, 577, 99, 249, 511, 410, 796, 768, 390, 537, 88, 173, 670, 298, 176, 248, 795].
2024-08-24 14:16:30,950 - opfython.models.supervised — INFO — Classifier has been fitted.
2024-08-24 14:16:30,951 - opfython.models.supervised — INFO — Training time: 2.5225155353546143 seconds.
2024-08-24 14:16:30,952 - opfython.models.supervised — INFO — Predicting data ...
2024-08-24 14:16:33,239 - opfython.models.supervised — INFO — Data has been predicted.
2024-08-24 14:16:33,239 - opfython.models.supervised — INFO — Prediction time: 2.2868170738220215 seconds.
ACURÁCIA: 0.987261495091883


/home/iara/.virtualenvs/pos-graduaçao/lib/python3.8/site-packages/opfython/math/general.py:96: RuntimeWarning: invalid value encountered in divide
  errors[:, 1] /= counts


Com o otimizador teremos uma base melhorada tarzendo assim o benefício da melhora da acurácia do treinamento realizado em cima
da base separada com as melhores características

In [2]:
import opfython.math.general as g
import opfython.stream.splitter as s
from opfython.models.supervised import SupervisedOPF
from sklearn.datasets import load_digits

import opytimizer.math.random as r
from opytimizer import Opytimizer
from opytimizer.core import Function
from opytimizer.optimizers.boolean import BPSO
from opytimizer.spaces import BooleanSpace

# Loads dataset
digits = load_digits()

# Gathers samples and targets
X = digits.data
Y = digits.target

#Adding 1 to labels, i.e., OPF should have labels from 1+
Y += 1

#Splits data into training and testing data
X_train, X_test, Y_train, Y_test = s.split(X, Y, percentage=0.5, random_state=1)

#Build a new dataset with features selected
def supervised_opf_selection(opytimizer):
    features = opytimizer[:, 0].astype(bool)
    X_train_selected = X_train[:, features]
    X_test_selected = X_test[:, features]
    
    opf = SupervisedOPF(distance="log_squared_euclidean", pre_computed_distance=None)
    opf.fit(X_train_selected, Y_train)
    
    preds = opf.predict(X_test_selected)
    
    acc = g.opf_accuracy(Y_test, preds)
    
    return 1 - acc
    

n_agents = 5
n_variables = 64

params = {"c1": r.generate_binary_random_number(size=(n_variables, 1)),
          "c2": r.generate_binary_random_number(size=(n_variables, 1))
         }

space = BooleanSpace(n_agents, n_variables)
optimizer = BPSO(params)
function = Function(supervised_opf_selection)

opt = Opytimizer(space, optimizer, function)

opt.start(n_iterations=3)


2024-08-24 14:32:20,745 - opfython.stream.splitter — INFO — Splitting data ...
2024-08-24 14:32:20,746 - opfython.stream.splitter — DEBUG — X_1: (898, 64) | X_2: (899, 64) | Y_1: (898,) | Y_2: (899,).
2024-08-24 14:32:20,747 - opfython.stream.splitter — INFO — Data splitted.
2024-08-24 14:32:20,747 - opytimizer.spaces.boolean — INFO — Overriding class: Space -> BooleanSpace.
2024-08-24 14:32:20,750 - opytimizer.core.space — DEBUG — Agents: 5 | Size: (64, 1) | Lower Bound: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] | Upper Bound: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.] | Mapping: ['x0', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15', 'x

/home/iara/.virtualenvs/pos-graduaçao/lib/python3.8/site-packages/opfython/math/general.py:96: RuntimeWarning: invalid value encountered in divide
  errors[:, 1] /= counts


2024-08-24 14:32:23,495 - opfython.models.supervised — DEBUG — Prototypes: [834, 65, 757, 416, 66, 871, 742, 448, 731, 89, 292, 616, 407, 396, 434, 233, 591, 706, 859, 726, 501, 459, 507, 582, 465, 353, 872, 220, 515, 609, 104, 267, 57, 748, 648, 445, 795, 777, 668, 258, 588, 58, 799, 692, 347, 720, 572, 570, 508, 841, 717, 254, 256, 571, 440, 281, 266, 329, 48, 116, 452, 683, 6, 400, 348, 487, 203, 759, 796, 100, 302, 577, 99, 88, 635, 644, 344, 54, 144, 165, 102, 204, 298, 124, 112, 317, 458, 477, 462, 768, 29, 547, 600, 537, 248, 787, 80, 270, 74, 444, 641, 563].
2024-08-24 14:32:24,150 - opfython.models.supervised — INFO — Classifier has been fitted.
2024-08-24 14:32:24,151 - opfython.models.supervised — INFO — Training time: 1.0496845245361328 seconds.
2024-08-24 14:32:24,152 - opfython.models.supervised — INFO — Predicting data ...
2024-08-24 14:32:25,034 - opfython.models.supervised — INFO — Data has been predicted.
2024-08-24 14:32:25,035 - opfython.models.supervised — INFO — P

  0%|          | 0/3 [00:00<?, ?it/s]

2024-08-24 14:32:30,785 - opfython.models.supervised — INFO — Overriding class: OPF -> SupervisedOPF.
2024-08-24 14:32:30,785 - opfython.core.opf — INFO — Creating class: OPF.
2024-08-24 14:32:30,786 - opfython.core.opf — DEBUG — Distance: log_squared_euclidean | Pre-computed distance: False.
2024-08-24 14:32:30,786 - opfython.core.opf — INFO — Class created.
2024-08-24 14:32:30,786 - opfython.models.supervised — INFO — Class overrided.
2024-08-24 14:32:30,787 - opfython.models.supervised — INFO — Fitting classifier ...
2024-08-24 14:32:30,790 - opfython.models.supervised — DEBUG — Finding prototypes ...
2024-08-24 14:32:31,177 - opfython.models.supervised — DEBUG — Prototypes: [724, 65, 853, 333, 796, 410, 249, 511, 854, 193, 616, 199, 203, 759, 337, 859, 369, 885, 434, 731, 659, 398, 95, 524, 852, 458, 537, 88, 862, 368, 173, 670, 17, 481, 491, 28, 882, 633, 727, 335, 714, 299, 748, 639, 820, 523, 576, 113, 547, 501, 726, 540, 565, 2, 281, 83, 400, 469, 789, 25, 648, 254, 347, 251, 6

...

2024-08-24 14:32:40,669 - opfython.models.supervised — INFO — Overriding class: OPF -> SupervisedOPF.
2024-08-24 14:32:40,669 - opfython.core.opf — INFO — Creating class: OPF.
2024-08-24 14:32:40,670 - opfython.core.opf — DEBUG — Distance: log_squared_euclidean | Pre-computed distance: False.
2024-08-24 14:32:40,670 - opfython.core.opf — INFO — Class created.
2024-08-24 14:32:40,670 - opfython.models.supervised — INFO — Class overrided.
2024-08-24 14:32:40,671 - opfython.models.supervised — INFO — Fitting classifier ...
2024-08-24 14:32:40,673 - opfython.models.supervised — DEBUG — Finding prototypes ...
2024-08-24 14:32:41,070 - opfython.models.supervised — DEBUG — Prototypes: [834, 65, 576, 113, 22, 38, 668, 821, 107, 217, 10, 879, 540, 36, 275, 608, 189, 883, 60, 190, 485, 644, 648, 157, 544, 872, 619, 172, 609, 104, 724, 66, 502, 727, 316, 470, 422, 126, 679, 592, 838, 845, 73, 571, 339, 185, 440, 116, 452, 683, 6, 203, 757, 144, 750, 577, 489, 99, 732, 745, 515, 220, 768, 390, 819

 67%|######6   | 2/3 [00:19<00:09,  9.74s/it, fitness=0.0103]

2024-08-24 14:32:50,308 - opfython.models.supervised — INFO — Overriding class: OPF -> SupervisedOPF.
2024-08-24 14:32:50,309 - opfython.core.opf — INFO — Creating class: OPF.
2024-08-24 14:32:50,310 - opfython.core.opf — DEBUG — Distance: log_squared_euclidean | Pre-computed distance: False.
2024-08-24 14:32:50,310 - opfython.core.opf — INFO — Class created.
2024-08-24 14:32:50,311 - opfython.models.supervised — INFO — Class overrided.
2024-08-24 14:32:50,311 - opfython.models.supervised — INFO — Fitting classifier ...
2024-08-24 14:32:50,314 - opfython.models.supervised — DEBUG — Finding prototypes ...
2024-08-24 14:32:50,688 - opfython.models.supervised — DEBUG — Prototypes: [664, 892, 860, 197, 628, 114, 447, 574, 834, 450, 678, 706, 95, 22, 65, 109, 439, 454, 19, 250, 487, 603, 666, 729, 768, 506, 390, 193, 379, 720, 804, 814, 643, 536, 185, 661, 635, 8, 300, 845, 606, 12, 855, 540, 568, 529, 791, 318, 410, 796, 176, 511, 655, 749, 398, 588, 40, 74, 759, 434, 728, 269, 474, 614, 2

100%|##########| 3/3 [00:37<00:00, 12.45s/it, fitness=0.0103]

2024-08-24 14:33:08,143 - opytimizer.opytimizer — INFO — Optimization task ended.
2024-08-24 14:33:08,144 - opytimizer.opytimizer — INFO — It took 47.38325309753418 seconds.
